<div align="center">
  <img src="https://upload.wikimedia.org/wikipedia/pt/a/a2/Brasao_UFPA.jpg" width="300">

  # **Universidade Federal do Pará**
  ## **Instituto de Tecnologia**
  ### **Faculdade de Engenharia de Computação e Telecomunicações**

  ---

  # **Disciplina:** Inteligência Computacional
  ### **Professor:** Dr. Aldebaro Barreto da Rocha Klautau Jr.

  ---

  ## 📑 **Tema:** Aplicações Básicas de *Recuperação Aumentada por Geração* (RAG)

</div>

**Data:** 12 de Fevereiro de 2026  

---

### **Resumo do Notebook**


**Pacote necessário:**

[Milvus](https://milvus.io/) é um banco de dados vetorial de código aberto popular que impulsiona aplicações de IA com busca de similaridade vetorial altamente performática e escalável. Neste tutorial, mostraremos como construir um pipeline RAG (Retrieval-Augmented Generation - Geração Aumentada por Recuperação) com Hugging Face e Milvus.

O sistema RAG combina um sistema de recuperação com um LLM (Learning Learning Machine - Máquina de Aprendizagem de Liderança). O sistema primeiro recupera documentos relevantes de um corpus usando o banco de dados vetorial Milvus e, em seguida, usa um LLM hospedado no Hugging Face para gerar respostas com base nos documentos recuperados.

## **1 - Preparação**

### **1.1 - Dependências e Ambiente**

In [10]:
!pip install --upgrade sentence-transformers huggingface-hub langchain_community langchain-text-splitters pypdf tqdm

In [11]:
!pip install -U pymilvus milvus-lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 6.8 MB/s eta 0:00:00


Além disso, recomendamos que você configure seu [Token de Acesso de Usuário do Hugging Face](https://huggingface.co/docs/hub/security-tokens) e o defina em suas variáveis ​​de ambiente, pois usaremos um LLM do Hugging Face Hub. Você poderá ter um limite baixo de solicitações se não configurar a variável de ambiente do token.

In [ ]:
import os

os.environ["HF_TOKEN"] = "hf_..."

### **1.2 - Preparar os dados**

Utilizamos o [AI Act PDF](https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf), um quadro regulamentar para IA com diferentes níveis de risco correspondentes a mais ou menos regulamentação, como conhecimento privado em nosso RAG.

In [2]:
%%bash

if [ ! -f "The-AI-Act.pdf" ]; then
    wget -q https://artificialintelligenceact.eu/wp-content/uploads/2021/08/The-AI-Act.pdf
fi

Utilizamos o [`PyPDFLoader`](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/) do LangChain para extrair o texto do PDF e, em seguida, dividimos o texto em partes menores. Por padrão, definimos o tamanho da parte como 1000 e a sobreposição como 200, o que significa que cada parte terá aproximadamente 1000 caracteres e a sobreposição entre duas partes será de 200 caracteres.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("The-AI-Act.pdf")
docs = loader.load()
print(len(docs))

108


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

In [5]:
text_lines = [chunk.page_content for chunk in chunks]

In [6]:
text_lines[0]

'EN   EN \n \n \n \nEUROPEAN \nCOMMISSION  \nBrussels, 21.4.2021  \nCOM(2021) 206 final \n2021/0106 (COD) \n \nProposal for a \nREGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL \nLAYING DOWN HARMONISED RULES ON ARTIFICIAL INTELLIGENCE \n(ARTIFICIAL INTELLIGENCE ACT) AND AMENDING CERTAIN UNION \nLEGISLATIVE ACTS \n{SEC(2021) 167 final} - {SWD(2021) 84 final} - {SWD(2021) 85 final}'

### **1.3 - Preparar o Modelo de Incorporação**

Defina uma função para gerar embeddings de texto. Usamos o [modelo de embedding BGE](https://huggingface.co/BAAI/bge-small-en-v1.5) como exemplo, mas você pode usar qualquer modelo de embedding, como os encontrados no [ranking MTEB](https://huggingface.co/spaces/mteb/leaderboard).

In [7]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")

def emb_text(text):
    return embedding_model.encode([text], normalize_embeddings=True).tolist()[0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Gere um vetor de teste e imprima suas dimensões e os primeiros elementos.

In [8]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

384
[-0.07660680264234543, 0.02531673200428486, 0.012505538761615753, 0.004595177713781595, 0.025780003517866135, 0.038167089223861694, 0.08050810545682907, 0.0030353753827512264, 0.02439218945801258, 0.004880349617451429]


## **2 - Carregar dados no Milvus**

### **2.1 - Criar a Coleção**

In [12]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./hf_milvus_demo.db")
collection_name = "rag_collection"
print("Conectado com sucesso!")

Conectado com sucesso!


> Quanto ao argumento de `MilvusClient`:
> - Definir o `uri` como um arquivo local, por exemplo, `./hf_milvus_demo.db`, ​​é o método mais conveniente, pois utiliza automaticamente o [Milvus Lite](https://milvus.io/docs/milvus_lite.md) para armazenar todos os dados neste arquivo.
> - Se você tiver uma grande quantidade de dados, digamos, mais de um milhão de vetores, você pode configurar um servidor Milvus mais eficiente no [Docker ou Kubernetes](https://milvus.io/docs/quickstart.md). Nessa configuração, use o URI do servidor, por exemplo, `http://localhost:19530`, como seu `uri`.

> - Se você quiser usar o [Zilliz Cloud](https://zilliz.com/cloud), o serviço de nuvem totalmente gerenciado para Milvus, ajuste o `uri` e o `token`, que correspondem ao [Endpoint Público e chave de API](https://docs.zilliz.com/docs/on-zilliz-cloud-console#cluster-details) no Zilliz Cloud.

---

**Atenção:** Como o disco do Colab é temporário, se você fechar a aba e voltar amanhã, esse arquivo `.db` terá sumido. Se você estiver guardando dados importantes, recomendo salvar esse arquivo no seu Google Drive montado:

```python
from google.colab import drive
drive.mount('/content/drive')

# Use um caminho dentro do seu Drive para não perder os dados
milvus_client = MilvusClient(uri="/content/drive/MyDrive/milvus_demo.db")
```

---

Verifique se a coleção já existe e, se existir, remova-a.

In [13]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

Crie uma nova coleção com os parâmetros especificados.

Se não especificarmos nenhuma informação de campo, o Milvus criará automaticamente um campo `id` padrão para a chave primária e um campo `vector` para armazenar os dados do vetor. Um campo JSON reservado é usado para armazenar campos não definidos no esquema e seus valores.

In [14]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_level="Strong",
)

### **2.2 - Inserir dados**

Percorra as linhas de texto, crie embeddings e insira os dados no Milvus.

Aqui está um novo campo `text`, que não está definido no esquema da coleção. Ele será adicionado automaticamente aos campos dinâmicos JSON reservados, podendo ser tratado como um campo normal em um nível mais alto.

In [15]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

insert_res = milvus_client.insert(collection_name=collection_name, data=data)
insert_res["insert_count"]

Creating embeddings: 100%|██████████| 424/424 [01:54<00:00,  3.71it/s]


424

## **3 - Construir RAG**

### **3.1 - Recuperar dados para uma consulta**

Vamos especificar uma pergunta a ser feita sobre o corpus.

In [46]:
question = "What is the legal basis for the proposal?"

Pesquise a pergunta na coleção e recupere as 3 principais correspondências semânticas.

In [47]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        emb_text(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

Vamos analisar os resultados da pesquisa:


In [48]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "EN 6  EN \n2. LEGAL BASIS, SUBSIDIARITY AND PROPORTIONALITY \n2.1. Legal basis \nThe legal basis for the proposal is in the first place Article 114 of the Treaty on the \nFunctioning of the European Union (TFEU), which provides for the adoption of measures to \nensure the establishment and functioning of the internal market.  \nThis proposal constitutes a core part of the EU digital single market strategy. The primary \nobjective of this proposal is to ensure the proper functioning of the internal market by setting \nharmonised rules in particular on the development, placing on the Union market and the use \nof products and services making use of AI technologies or provided as stand -alone AI \nsystems. Some Member States are already considering national rules to ensure that AI is safe \nand is developed and used in compliance with fundamental rights obligations. This will likely \nlead to two main problems: i) a fragmentation of the internal market on essential elemen

### **3.2 - Use LLM para obter uma resposta RAG**

Antes de compor o prompt para o LLM, vamos primeiro transformar a lista de documentos recuperada em uma string simples.

In [49]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

Defina os prompts para o Modelo de Linguagem. Este prompt é montado com os documentos recuperados do Milvus.

In [51]:
PROMPT = """
Utilize as informações a seguir, entre tags <context>, para responder à pergunta entre tags <question>.
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [52]:
messages = [
    {
        "role": "system",
        "content": "Você é um assistente acadêmico. Responda em Português (Brasil) com base no contexto fornecido."
    },
    {
        "role": "user",
        "content": PROMPT.format(context=context, question=question)
    }
]

Utilizamos o modelo [Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) da Meta, acessado via API de Inferência da Hugging Face, para gerar respostas baseadas em um sistema de mensagens estruturadas (Chat Completion).

In [53]:
from huggingface_hub import InferenceClient

repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"

llm_client = InferenceClient(model=repo_id, timeout=120)

Finalmente, podemos formatar o enunciado e gerar a resposta.

In [58]:
try:
    # O chat_completion enviará automaticamente o seu HF_TOKEN configurado
    response = llm_client.chat_completion(
        messages=messages,
        max_tokens=1000,
        temperature=0.3
    )
    answer = response.choices[0].message.content.strip()
    print("--- RESPOSTA ---")
    print(answer)
except Exception as e:
    print(f"Erro ao chamar a API: {e}")

--- RESPOSTA ---
A base legal para a proposta está, em primeiro lugar, no Artigo 114 do Tratado sobre o Funcionamento da União Europeia (TFEU), que prevê a adoção de medidas para garantir a estabelecimento e o funcionamento do mercado interior.


Parabéns! Você criou um pipeline RAG com Hugging Face e Milvus.